In [1]:
import pandas as pd
import numpy as np
from tqdm import tqdm

import matplotlib.pyplot as plt
import seaborn as sns

from sklearn.model_selection import train_test_split
# from sklearn.feature_extraction.text import CountVectorizer
from sklearn.preprocessing import OneHotEncoder
from sklearn.linear_model import LinearRegression
from sklearn.linear_model import LogisticRegression

from sklearn.metrics import mean_squared_error

dat = pd.read_pickle('fenty_data_cleaned.pkl')



In [2]:
reviewer_features = ['hair_color', 'eye_color', 'skin_tone', 'skin_type']

X = dat[reviewer_features]
y = dat['rating']

train_X, val_X, train_y, val_y = train_test_split(X, y,test_size=0.3, random_state=0)

In [3]:
enc = OneHotEncoder(sparse=False)
train_X_transform = enc.fit_transform(train_X)

In [4]:
train_X_transform

array([[0., 0., 0., ..., 1., 0., 0.],
       [0., 0., 0., ..., 1., 0., 0.],
       [0., 0., 1., ..., 0., 0., 0.],
       ...,
       [0., 0., 0., ..., 0., 0., 0.],
       [0., 0., 0., ..., 0., 0., 1.],
       [0., 0., 1., ..., 0., 0., 0.]])

In [5]:
reg = LinearRegression().fit(train_X_transform, train_y)

In [6]:
val_X_transform = enc.transform(val_X)

In [7]:
reg.score(val_X_transform, val_y)

0.017726269821426888

In [8]:
predict_y = reg.predict(val_X_transform)

In [9]:
print('MSE: ', mean_squared_error(val_y, predict_y))
print('RMSE: ', np.sqrt(mean_squared_error(val_y, predict_y)))

MSE:  1.8105026661220962
RMSE:  1.3455492061318666


### trying out classification model